## <font color="blue"> Data</font>

In [2]:
using DataFrames
using JWAS.Datasets
using JWAS.MT

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### Genetic covariance matrix and residual covariance matrix

In [4]:
R      = [10.0 0.0
           0.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [5]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [6]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [7]:
model3    = MT.build_model(model_equations,R);

In [8]:
MT.set_covariate(model3,"age")

In [9]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/Github/JWAS.jl/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


In [10]:
Pi=Dict([1.0; 1.0]=>0.1,[0.0; 0.0]=>0.9)
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50,estimatePi=false,constraint=true,
                  printout_frequency=10,methods="BayesCC",output_marker_effects_frequency=1);

Posterior means at iteration: 10
Residual covariance matrix: 
[131.72359988521032 0.0
 0.0 4.644832278326033]
Marker effects covariance matrix: 
[11.061939906664731 1.051357805627184
 1.051357805627184 1.1809885436033594]

Posterior means at iteration: 20
Residual covariance matrix: 
[132.23078898731558 0.0
 0.0 5.477737573520423]
Marker effects covariance matrix: 
[8.365613191962364 0.5695088645509314
 0.5695088645509314 1.8118860265288093]

Posterior means at iteration: 30
Residual covariance matrix: 
[126.14887563973927 0.0
 0.0 5.734876868969988]
Marker effects covariance matrix: 
[7.475446282619975 0.3385129671551335
 0.3385129671551335 1.3625973599245875]

Posterior means at iteration: 40
Residual covariance matrix: 
[118.17584003668372 0.0
 0.0 5.519224567760144]
Marker effects covariance matrix: 
[8.521169814463773 -0.0856759564678628
 -0.0856759564678628 1.5109361726944726]

Posterior means at iteration: 50
Residual covariance matrix: 
[104.86976690786469 0.0
 0.0 4.5092181802

In [11]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

In [12]:
out2["posterior mean of marker effects"]

2-element Array{Any,1}:
 5x2 Array{Any,2}:
 "x1"   2.79339   
 "x2"   0.416431  
 "x3"   0.00157184
 "x4"   0.0251137 
 "x5"  -0.0162638 
 5x2 Array{Any,2}:
 "x1"   1.68362   
 "x2"   0.0606433 
 "x3"  -0.0222896 
 "x4"   0.0542484 
 "x5"  -0.00136889

In [13]:
using JWAS.QTL

In [14]:
file1="marker_effects_BW_2016-07-27T00:07:40.txt"
file2="marker_effects_CW_2016-07-27T00:07:40.txt";

In [15]:
a,b,c=get_additive_genetic_variances(model3,file1,file2);

In [16]:
get_breeding_values(model3,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -7.07086 │ 1.91292 │
│ 2   │ "D1" │ 3.44448  │ 8.45197 │
│ 3   │ "O1" │ 20.5108  │ 17.237  │
│ 4   │ "O3" │ -16.8844 │ 4.52614 │      
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV      │
├─────┼──────┼──────────┼──────────┤
│ 1   │ "S1" │ -1.28609 │ 0.133655 │
│ 2   │ "D1" │ 1.53838  │ 0.619756 │
│ 3   │ "O1" │ 3.74108  │ 1.09428  │
│ 4   │ "O3" │ -3.99336 │ 0.651    │